Hiii!! , this is my first kaggle notebook and i am quite excited to share my ideas with the community, being a newbie i might do some cranky stuff below, sorry for it :) , thanks for providing me with this oppurtunity.

# **IMPORTING OUR DATA:**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('../input/public-tenders-romania-20072016/contracts.csv')

# FIRST IMPRESSIONS:

In [ ]:
data.head()

A initial look of the data can provide us with some insights to proceed with.First lets try to get some quantitative and qualitative understanding about missing data points so that we can come up with ideas to clean it up.

In [ ]:
missing_data_percentage =  data.isnull().sum().sum()/np.product(data.shape)*100
print("Missing Data: ",missing_data_percentage,"%")

**OBSERVATION**: A whooping 25 % of our data cells are unfilled. 

While it may sound bad , this is where we must question the context of the missing data,we must answer a few questions like:
 
 1. In the context of that particular column, does NaN mean something else?
 
 2. is it just a missing valuable data point?
 
 3. Why is that particular data point unfilled?
 
 
The answers to these questions will be the clue for us on what steps to follow when we actually start working with the data.

# Cleaning up the Data:

# In the context of that particular column, does NaN mean something else?

**YES:**  In this Dataset the columns that contain information about Yes/No type questions refers a Yes as *DA* and a No as *NaN* ,so let us try to remove those null entries by replacing such columns with apt boolean data types.  

Lets start by converting the yes/no type series to a boolean series by replacing **DA/NAN  as  True/False**

In [ ]:

data['EU_Funds'].replace(np.NaN,False,inplace =True)
data['EU_Funds'].replace('DA',True,inplace =True)

data['Periodic_Contract'].replace(np.NaN,False,inplace =True)
data['Periodic_Contract'].replace('DA',True,inplace =True)

data['Subcontracted'].replace(np.NaN,False,inplace =True)
data['Subcontracted'].replace('DA',True,inplace =True)

data['With_Electronic_Auction'].replace(np.NaN,False,inplace =True)
data['With_Electronic_Auction'].replace('DA',True,inplace =True)


In [ ]:
missing_data_percentage =  data.isnull().sum().sum()/np.product(data.shape)*100
print("Missing Data: ",missing_data_percentage,"%")

That has reduced our null entry  by around **10%** , not only that but it also has made the columns boolean based which might turn out to be hugely helpful when it comes to data analysis

In [ ]:
data.head()

In [ ]:
data.isnull().sum().sort_values(ascending=False).head(15)

# is it just a missing valuable data point?

Some of our columns fall in this category too , they are just plain unfilled data point which might be extremly valuable during data analysis,
to get the most out of this dataset it becomes essential for us to find ways to extract and fill in those missing data points. For instance the Winner Country tells us about the country of the company which won the tender offer , around 79733 datapoints about Winner country is missing ,it is so huge that it can make a considerable amount of difference when it comes to data analytics.

Before we start with this idea it is important for us to procure the dataframe and remove inconsistancies due to the way our data is structured
for ex a company named "aaaaa s.n" and "aaaaa S.N" are grouped as different entities , we will fist try to come up with a solution for this.

**Plan:** To try to extract raw data from columns so that Things like spaces,case,punctuations dont make a difference when we group data

In [ ]:
missing_data_due_to_winner_country_percentage =  data.isnull().sum()["Winner_Country"]/np.product(data.shape)*100
print("Missing Data: ",missing_data_due_to_winner_country_percentage,"%")

In [ ]:
data["Winner_Country"].replace(np.NaN,"__",inplace =True)
data["Winner_Country"].value_counts()

We have replaced all the NaN s in the Winner_Country column by a placeHolder **__** so that it is not neglected when called upon by some pandas fxn's  

In [ ]:
data["Winner_City"].replace(np.NaN,"NO_CITY_STAT",inplace =True)
data["Winner_City"].replace("-","NO_CITY_STAT",inplace =True)

We have also replaced Nan entries in Winner_City column by **NO_CITY_STAT** for similar reasons

In [ ]:

display(data["Winner_City"].value_counts().head(20))

In [ ]:
import string
data["Raw_City_Data"] = data["Winner_City"].str.strip().str.translate(str.maketrans('', '', string.punctuation)).str.replace(" ","").str.upper()

display(data["Raw_City_Data"].value_counts().head(20))

In [ ]:
print("Relative change in size of group BUCURESTI:",(351435-333776)/333776*100,"%")

By doing a little bit of string manipulation, we have removed all the spaces,punctuations and special characters from the Winner_City column 
doing this we get what i would call as a **RAW DATA POINT** which conserves useful data but lacks the luxury of being a well punctuated and 
spaced string, this raw datapoint removes problems like case sensitivity,equality of city name being disrupted by things like spaces, commas ,periods.

And as we see above , we notice how Bucuresti a city in Romania had **333776** entries in our first output , but when we group them in terms of RAWDATA we see that BUCURESTI has 351435 entries , around **20000** datapoints was being missed out in the first due to complications byspaces,case sensitivity and punctuation that accounts for a relative change of around **5%**

We will do something similar to the columns winner and contract_type

In [ ]:
data["Raw_Winner_Data"] = data["Winner"].str.strip().str.translate(str.maketrans('', '', string.punctuation)).\
str.replace(" ","").str.upper()

In [ ]:
display(data["Winner"].value_counts()[:100])

In [ ]:
display(data["Raw_Winner_Data"].value_counts().head(20))

In [ ]:
data["Raw_Contract_Title"] = data["Contract_Title"].str.strip().str.translate(str.maketrans('', '', string.punctuation)).\
str.replace(" ","").str.upper()

In [ ]:
display(data["Contract_Title"].value_counts().head())

In [ ]:
display(data["Raw_Contract_Title"].value_counts().head())

In [ ]:
print("Relative change for MEDICAMENTE:",(14627-7188)/7128*100)

Thats is a whooping **Relative change of 104%** , this is by far one of the best examples on why this approach helps
this approach can be extended to many such columns in our dataframe , so that a more cured version of the dataset can be extracted 

Now that we have made our database more grouping friendly we can start adressing the null problem . to start with lets take the Winners_Country column which has 79733 null points and let us try to come up with an idea to fill in these spaces.

**Plan:** we can collect  the city data of data points with missing country, and check if there exists a entry with country value which has the same city value so that we can fill in the country of the missing value.

In [ ]:
data["Winner_Country"].value_counts().head(20)

In [ ]:
list_of_countries = list(data["Winner_Country"].unique())
list_of_countries.remove("__")

In [ ]:
for x in list_of_countries:
    x_cities = list(data[data["Winner_Country"] == x]["Raw_City_Data"].unique())
    if "NOCITYSTAT" in x_cities:
        x_cities.remove("NOCITYSTAT")
    data.loc[(data["Winner_Country"]=="__") & (data["Raw_City_Data"].isin(x_cities)),"Winner_Country"] = x

In [ ]:
display(data["Winner_Country"].value_counts())

In [ ]:
print("Relative change in Null values of Winner_Country",(-1203+79733)/79733*100 , "%")

We have reduced the no of null points by a whooooooping **98.5%** , from having around **79000 null points** , we now have only **1203 null points**,we have by comparing and extracting data from other columns **added another 78000 datapoints** to our data frame , this added datapoints are not random but appropriate datapoints which enriches our dataframe.

lets check if we can do something about any other columns

In [ ]:
data.isnull().sum().sort_values(ascending=False).head(15)

If we look up we can observe how the column **EU_Fund** is **number 1** in our list , to understand why it is so , it is important for us to look into another data column called as the **EU_Funds** which is a **boolean column** saying wFlag to indicate if the tender is about accessing EU Funds, it is a yes/no boolean column on the other hand  the column EU_Fund says about that specific fund name which was used.

In [ ]:
data["EU_Funds"].value_counts()

We can see how around 860000 entries of data points did not use a EU fund and hence those columns are left empty , so all we want to do is to check if the tender uses a EU fund and incase if it is False we can replace the null Value with a string like **"NO EU Funds Used"**

In [ ]:
data.loc[~data["EU_Funds"],"EU_Fund"] = "NO EU FUNDS USED"

In [ ]:
data.isnull().sum().sort_values(ascending=False).head(15)

In [ ]:
print("Relative change in no of null points for EU funds: ",(875511-15991)/875511*100,"%")

We have reduced the no of null entries in EU_Funds col by 98.17% and filled it with good informative data.

While this may sound fancy,what we have essentialy did is just replace NaN s with I Dont Know, while this might provide some extra layer of info, to do this or not will be completley an individuals choice. The idea of using other columns to extract missing values cannot be extended to fields like Financing_Type,Contracting_Authority_Type,Garantee_Deposits ... and so on , Filling them with I dont know's will not be a much of a improvement to the quality of our dataset.

The series of columns  Participation_Announcement_Number,Participation_Announcement_Date,Participation_Estimated_Value, Participation Estimated_Value_Currency are some of the least data rich field in the dataset, they are also the ones with consiting data which cannot be filled in by looking into other columns

# CPV CODES:

INFO FROM [Wikipedia](http://en.wikipedia.org/wiki/Common_Procurement_Vocabulary)

# INTRODUCTION:

The Common Procurement Vocabulary (CPV) has been developed by the European Union to facilitate the processing of invitations to tender published in the Official Journal of the European Union (OJEU) by means of a single classification system to describe the subject matter of public contracts. It was established by Regulation (EC) No 2195/2002 of the European Parliament and of the Council on the Common Procurement Vocabulary (CPV) and amended by European Commission Regulation (EU) No. 213/2008  issued on 28 November 2007.

# STRUCTURE:
CPV codification consists of a main vocabulary which defines the subject of the contract, and a supplementary vocabulary to add further qualitative information. The main vocabulary is based on a tree structure made up with codes of up to 9 digits (an 8 digit code plus a check digit). This combination of digits is associated with a wording that describes the type of supplies, works or services defining the subject of the contract. A Call for Tender is quite often described by more than one CPV Code, aiming to give a better and more detailed description of the object of the contract. Commercial organisations promoting public contracts to their members or readers generally use CPV codes to identify business sectors likely to be interested in specific tenders, along with NUTS Codes which indicate the country and region within which the contract is to be performed.

Main Vocabulary classification structure

The numerical code consists of 8 digits, subdivided into:

Divisions: first two digits of the code XX000000-Y.

Groups: first three digits of the code XXX00000-Y.

Classes: first four digits of the code XXXX0000-Y.

Categories: first five digits of the code XXXXX000-Y.

# REMARKS:

The CPV_Code column in our data is by far the most important part of the dataset doing a bit of research on how cpv codes work, we come to know how much data rich the codes are , they supply a plethora of information , sadly our dataset doesnot use the power of csv code to its fullest 

# Extending the Dataset:

From [Here](http://simap.ted.europa.eu/cpv)
we can get a xls file containing a list of cpv codes and thier defenitions but when i did this first time i found out a few codes missing from the xls file which was found in a pdf file from the same site so i went on  to write a python script to extract data from that pdf file and wrote it back in a good csv form so that it can be used effectively

In [ ]:
code = pd.read_excel('../input/cpv-codes/code2003.xls')

In [ ]:
code.columns = ["2003_CODE","2003_DESC","2007_CODE","2007_DESC","q","w","e","r","t","y","u","i","o"]
code.drop(columns=["q","w","e","r","t","y","u","i","o"],inplace=True)
code

Looks like our excel data requires a bit of cleanup

In [ ]:
stat_2003 = code[["2003_CODE","2003_DESC"]].copy()
stat_2003.dropna(inplace=True)
stat_2007 = code[["2007_CODE","2007_DESC"]].copy()
stat_2007.dropna(inplace=True)
stat_2007.drop_duplicates(subset=['2007_CODE'],inplace=True)

In [ ]:
stat_2003.columns = ["CODE","DESC"]
stat_2007.columns = ["CODE","DESC"]
code_stats = pd.concat([stat_2003, stat_2007], ignore_index=False, sort=False)
code_stats.drop_duplicates(subset=['CODE'],inplace=True)
info = code_stats[code_stats["CODE"].str.contains("000000")].copy()
info["KEY"] = ""
info["KEY"] = info["CODE"].str[:2]
info.drop_duplicates(subset=['KEY'],inplace=True)
info.set_index("KEY",inplace = True)
div_dict = info.to_dict('index')

We have merged both **2003 and 2007** cpv codes into one single dataframe which has entries of the form **XX000000** , which describes the division of the tender we have also converted it to a dictionary which returns the division description when passed on the code.

In [ ]:
data.dropna(how='all',inplace=True)

In [ ]:
def find_division(code):
        if len(code)>2:
            u = str(div_dict.get(code[:2])['DESC'])
            return str(u)


We have created a fxn that gets a key and returns the appropriate value from our dictionary,We will now create a new column called the **Division**  which will give us the division of that particular tender.

In [ ]:
data["Division"] = ""

In [ ]:
data["Division"] =  data.apply(lambda x:find_division(x["CPV_Code"]),axis=1)

In [ ]:
data["Division"].value_counts()

Now we have added the first layer of data to our data frame extracted from the cpv codes, a division gives us a broad idea about our tender. similarly we will create groups , who are more descriptive and specific than a division, and so on we will create a class and then a category column too

In [ ]:
info.reset_index(inplace=True)

In [ ]:
info = code_stats[code_stats["CODE"].str.contains("00000")].copy()
info["KEY"] = ""
info["KEY"] = info["CODE"].str[:3]
info.drop_duplicates(subset=['KEY'],inplace=True)
info.set_index("KEY",inplace = True)
group_dict = info.to_dict('index')

In [ ]:
def find_grp(code):
    if len(code)>3:
        u = str(group_dict.get(code[:3])['DESC'])
        if u is not None:
            return u
        else:
            return "None"

In [ ]:
data["Group"] = ""

In [ ]:
data["Group"] =  data.apply(lambda x:find_grp(x["CPV_Code"]),axis=1)

In [ ]:
data["Group"].value_counts()

In [ ]:
#Code for creating a class column

info = code_stats[code_stats["CODE"].str.contains("0000")].copy()

info["CODE"] = info["CODE"].str[:4]
info.drop_duplicates(subset=['CODE'],inplace=True)
info.set_index("CODE",inplace = True)
class_dict = info.to_dict('index')

def find_class(code):
    if len(code)>3:
        u = class_dict.get(code[:4])
        if u is not None:
            return u["DESC"]
        else:
            return "None"

data["Class"] = "_"
data["Class"] =  data.apply(lambda x:find_class(x["CPV_Code"]),axis=1)

In [ ]:
#code for creating a category column

info = code_stats[code_stats["CODE"].str.contains("000")].copy()

info["CODE"] = info["CODE"].str[:5]
info.drop_duplicates(subset=['CODE'],inplace=True)
info.set_index("CODE",inplace = True)
categ_dict = info.to_dict('index')

def find_categ(code):
    if len(code)>3:
        u = categ_dict.get(code[:5])
        if u is not None:
            return u["DESC"]
        else:
            return "None"

data["Category"] = "_"
data["Category"] =  data.apply(lambda x:find_class(x["CPV_Code"]),axis=1)

And to end it all we will add a final level of data which is the most specific of all we will call it **CPV_Data**, this will contain the most specific descriptive details of the tender,when i did it for the first time i found out that the xls file which we used missed a few data points, so i scrapped of data from pdf file in the same site wrote a script to change it to csv and we will now use that here 

In [ ]:
stat_2003 = code[["2003_CODE","2003_DESC"]].copy()
stat_2003.dropna(inplace=True)

stat_2007 = code[["2007_CODE","2007_DESC"]].copy()
stat_2007.dropna(inplace=True)

stat_2007.drop_duplicates(subset=['2007_CODE'],inplace=True)
stat_2007.set_index("2007_CODE",inplace=True)

stat_2003.set_index("2003_CODE",inplace=True)


dict_2003 = stat_2003.to_dict('index')
dict_2007 = stat_2007.to_dict('index')

code1 = set(data["CPV_Code"].unique())
code2 = set(stat_2007.index)
z = code1.intersection(code2)

def extract_stat(code):
    if code in z:
        u =  dict_2007.get(code)
        return u['2007_DESC']
    else:
        return "None"
    
    
data["CPV_Data"] = ""
data["CPV_Data"] =  data.apply(lambda x:extract_stat(x["CPV_Code"]),axis=1)
data["CPV_Data"].value_counts()

We have extracted data of 2007 based cpv codes from the excel sheet in a very similar fashion , we can see how we still miss around 62000 datapoints

In [ ]:
code1 = set(data["CPV_Code"].unique())
code2 = set(stat_2003.index)
z =code1.intersection(code2)
def second_extract_stat(code,data):
    if str(data) == "None":
        if code in z:
            v= str(dict_2003.get(code)['2003_DESC'])
            return v
        else:
            return str(data)
    else:
        return str(data)

In [ ]:
data["CPV_Data"] =  data.apply(lambda x:second_extract_stat(x["CPV_Code"],x["CPV_Data"]),axis=1)

In [ ]:
data[data["CPV_Data"]=="None"]["CPV_Data"].value_counts()

after extracting the 2003 based cpv codes from our .xls file we can still see that we are missing around 1300 datapoints , this is where our pdf based .csv file comes in.

In [ ]:
cpv_code = pd.read_csv("../input/cpv-codes/code.csv")
cpv_code.set_index('CODE',inplace=True)
cpv_data_dict = cpv_code.to_dict('index')

code1 = set(data["CPV_Code"].unique())
code2 = set(cpv_code.index)
z =code1.intersection(code2)
def thirdExtract(code,data):
    if str(data) == "None":
        if code in z:
            v= str(cpv_data_dict.get(code)['DESC'])
            return v
        else:
            return str(data)
    else:
        return str(data)

    
data["CPV_Data"] =  data.apply(lambda x:thirdExtract(x["CPV_Code"],x["CPV_Data"]),axis=1)

In [ ]:
data[data["CPV_Data"]=="None"]["CPV_Data"].value_counts()

**WOW that is a 0% null entry column**

After reading in the csv file , we can see how no data point has None value to it , every single data point in our dataframe has a CPV Data to it, while fields like contract type are prepresent in the dataset there are around **18000 Null entries** to it, moreover the information in such field are incomparable to what is provided by the cpv codes

With columns like division,classes,categories we can not only analyse the dataset as a whole but also as seperate groups , for example we can now do a brief analysis on Tenders for the agriculture department,what % of the tenders were for rice,what is the avg value of tenders to **Uranium ores** and so on

Using this idea of cpv codes will enrich  our dataset with a plhetora of information and data

# WHATS UP NEXT?

there are lots of things that i have missed when it comes to cleaning the dataset , things like Winner_Adress has a lot of issues when it comes to data grouping, because of spaces,punctuation and case sensitivity, so maybe using the idea of RAWDATA will improve the data set,things like legislation ID are of little use as they are mostly Nulls,data scrapping a few UN/EU sites will enrich the dataset more than anything.


        "The greatest challenges humans face throw-out their lives are two"

                        1. the challenge of where to start

                        2. the challenge of when to stop. 


THANK YOUUUUU!!!!